https://huggingface.co/docs/smolagents/main/en/examples/rag

In [1]:
# Installation
!pip install smolagents
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/smolagents.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 5.0 MB/s eta 0:00:00


# Agentic RAG

## Introduction to Retrieval-Augmented Generation (RAG)

Retrieval-Augmented Generation (RAG) combines the power of large language models with external knowledge retrieval to produce more accurate, factual, and contextually relevant responses. At its core, RAG is about "using an LLM to answer a user query, but basing the answer on information retrieved from a knowledge base."

### Why Use RAG?

RAG offers several significant advantages over using vanilla or fine-tuned LLMs:

1. **Factual Grounding**: Reduces hallucinations by anchoring responses in retrieved facts
2. **Domain Specialization**: Provides domain-specific knowledge without model retraining
3. **Knowledge Recency**: Allows access to information beyond the model's training cutoff
4. **Transparency**: Enables citation of sources for generated content
5. **Control**: Offers fine-grained control over what information the model can access

### Limitations of Traditional RAG

Despite its benefits, traditional RAG approaches face several challenges:

- **Single Retrieval Step**: If the initial retrieval results are poor, the final generation will suffer
- **Query-Document Mismatch**: User queries (often questions) may not match well with documents containing answers (often statements)
- **Limited Reasoning**: Simple RAG pipelines don't allow for multi-step reasoning or query refinement
- **Context Window Constraints**: Retrieved documents must fit within the model's context window

## Agentic RAG: A More Powerful Approach

We can overcome these limitations by implementing an **Agentic RAG** system - essentially an agent equipped with retrieval capabilities. This approach transforms RAG from a rigid pipeline into an interactive, reasoning-driven process.

### Key Benefits of Agentic RAG

An agent with retrieval tools can:

1. ✅ **Formulate optimized queries**: The agent can transform user questions into retrieval-friendly queries
2. ✅ **Perform multiple retrievals**: The agent can retrieve information iteratively as needed
3. ✅ **Reason over retrieved content**: The agent can analyze, synthesize, and draw conclusions from multiple sources
4. ✅ **Self-critique and refine**: The agent can evaluate retrieval results and adjust its approach

This approach naturally implements advanced RAG techniques:
- **Hypothetical Document Embedding (HyDE)**: Instead of using the user query directly, the agent formulates retrieval-optimized queries ([paper reference](https://huggingface.co/papers/2212.10496))
- **Self-Query Refinement**: The agent can analyze initial results and perform follow-up retrievals with refined queries ([technique reference](https://docs.llamaindex.ai/en/stable/examples/evaluation/RetryQuery/))

## Building an Agentic RAG System

Let's build a complete Agentic RAG system step by step. We'll create an agent that can answer questions about the Hugging Face Transformers library by retrieving information from its documentation.

You can follow along with the code snippets below, or check out the full example in the smolagents GitHub repository: [examples/rag.py](https://github.com/huggingface/smolagents/blob/main/examples/rag.py).

### Step 1: Install Required Dependencies

First, we need to install the necessary packages:

```bash
pip install smolagents pandas langchain langchain-community sentence-transformers datasets python-dotenv rank_bm25 --upgrade
```

If you plan to use Hugging Face's Inference API, you'll need to set up your API token:

In [2]:
!pip install smolagents pandas langchain langchain-community sentence-transformers datasets python-dotenv rank_bm25 --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [3]:
# Load environment variables (including HF_TOKEN)
from dotenv import load_dotenv
load_dotenv()

False

### Step 2: Prepare the Knowledge Base

We'll use a dataset containing Hugging Face documentation and prepare it for retrieval:

### api

In [4]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

# Load the Hugging Face documentation dataset
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# Filter to include only Transformers documentation
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

# Convert dataset entries to Document objects with metadata
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

# Split documents into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Characters per chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],  # Priority order for splitting
)
docs_processed = text_splitter.split_documents(source_docs)

print(f"Knowledge base prepared with {len(docs_processed)} document chunks")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2647 [00:00<?, ? examples/s]

Knowledge base prepared with 14695 document chunks


### Step 3: Create a Retriever Tool

Now we'll create a custom tool that our agent can use to retrieve information from the knowledge base:

In [5]:
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        # Initialize the retriever with our processed documents
        self.retriever = BM25Retriever.from_documents(
            docs, k=10  # Return top 10 most relevant documents
        )

    def forward(self, query: str) -> str:
        """Execute the retrieval based on the provided query."""
        assert isinstance(query, str), "Your search query must be a string"

        # Retrieve relevant documents
        docs = self.retriever.invoke(query)

        # Format the retrieved documents for readability
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# Initialize our retriever tool with the processed documents
retriever_tool = RetrieverTool(docs_processed)

> [!TIP]
> We're using BM25, a lexical retrieval method, for simplicity and speed. For production systems, you might want to use semantic search with embeddings for better retrieval quality. Check the [MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) for high-quality embedding models.

### Step 4: Create an Advanced Retrieval Agent

Now we'll create an agent that can use our retriever tool to answer questions:

https://huggingface.co/docs/smolagents/main/en/examples/rag

In [13]:
from smolagents import InferenceClientModel, CodeAgent

# Initialize the agent with our retriever tool
agent = CodeAgent(
    tools=[retriever_tool],
    model=InferenceClientModel(model_id="meta-llama/Llama-3.2-3B-Instruct"),
    # List of tools available to the agent
    #model=InferenceClientModel(),  # Default model "Qwen/Qwen2.5-Coder-32B-Instruct"
    max_steps=4,  # Limit the number of reasoning steps
    verbosity_level=2,  # Show detailed agent reasoning
)

# To use a specific model, you can specify it like this:


> [!TIP]
> Inference Providers give access to hundreds of models, powered by serverless inference partners. A list of supported providers can be found [here](https://huggingface.co/docs/inference-providers/index).

### Step 5: Run the Agent to Answer Questions

Let's use our agent to answer a question about Transformers:

In [14]:
# Ask a question that requires retrieving information
question = "For a transformers model training, which is slower, the forward or the backward pass?"

# Run the agent to get an answer
agent_output = agent.run(question)

# Display the final answer
print("\nFinal answer:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ For a transformers model training, which is slower, the forward or the backward pass?                           │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Llama-3.2-3B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To answer this question, I need to understand the process of training a transformer model. The forward    
pass is used to generate the output of the model, while the backward pass is used to calculate the gradients of the
loss function with respect to the model's parameters. I will use the `retriever` tool to get information about the 
forward and backward passes in transformer model training.                                                         
                                                                                                                   
<code>                                                                                                             
forward_pass_info = retriever(query="transformer model training forward pass")                                     
backward_pass_info = retriever(query="transformer model training backward pass")                                   
print("Forward pass information:", forward_pass_info)                                                              
print("Backward pass information:", backward_pass_info)                                                            
</code>                                                                                                            
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  forward_pass_info = retriever(query="transformer model training forward pass")                                   
  backward_pass_info = retriever(query="transformer model training backward pass")                                 
  print("Forward pass information:", forward_pass_info)                                                            
  print("Backward pass information:", backward_pass_info)                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Forward pass information: 
Retrieved documents:


===== Document 0 =====
*Recent work in language modeling demonstrates that training large transformer models advances the state of the art
in
Natural Language Processing applications. However, very large models can be quite difficult to train due to memory
constraints. In this work, we present our techniques for training very large transformer models and implement a 
simple,
efficient intra-layer model parallel approach that enables training transformer models with billions of parameters.
Our

===== Document 1 =====
*Recent work in language modeling demonstrates that training large transformer models advances the state of the art
in
Natural Language Processing applications. However, very large models can be quite difficult to train due to memory
constraints. In this work, we present our techniques for training very large transformer models and implement a 
simple,
efficient intra-layer model parallel approach that enables training transformer models with billions of parameters.
Our

===== Document 2 =====
- A transformer model trained on several languages. There are three different type of training for this model and 
the library provides checkpoints for all of them:

===== Document 3 =====
Saving all activations from the forward pass in order to compute the gradients during the backward pass can result 
in 
significant memory overhead. The alternative approach of discarding the activations and recalculating them when 
needed 
during the backward pass, would introduce a considerable computational overhead and slow down the training process.

===== Document 4 =====
model_inputs = tokenizer(src_text, text_target=tgt_text, return_tensors="pt")

model(**model_inputs)  # forward pass
```

- Generation

===== Document 5 =====
Next, regarding the debugging strategy, there are generally a few from
which to choose from:

-   Decompose the original model into many small testable components and
    run a forward pass on each of those for verification
-   Decompose the original model only into the original *tokenizer* and
    the original *model*, run a forward pass on those, and use
    intermediate print statements or breakpoints for verification

===== Document 6 =====
Next, regarding the debugging strategy, there are generally a few from which to choose from:

- Decompose the original model into many small testable components and run a forward pass on each of those for
  verification
- Decompose the original model only into the original *tokenizer* and the original *model*, run a forward pass on
  those, and use intermediate print statements or breakpoints for verification

===== Document 7 =====
layer is falsely activated during the forward pass, *i.e.* pass *self.training* to [PyTorch's functional 
dropout](https://pytorch.org/docs/stable/nn.functional.html?highlight=dropout#torch.nn.functional.dropout)

===== Document 8 =====
Having managed to correctly load the pretrained weights into the 🤗 Transformers implementation, you should now 
make
sure that the forward pass is correctly implemented. In [Get familiar with the original 
repository](#34-run-a-pretrained-checkpoint-using-the-original-repository), you have already created a script that 
runs a forward
pass of the model using the original repository. Now you should write an analogous script using the 🤗 Transformers

===== Document 9 =====
4.  [ ] Created script that successfully runs forward pass using
    original repository and checkpoint

5.  [ ] Successfully opened a PR and added the model skeleton to Transformers

6.  [ ] Successfully converted original checkpoint to Transformers
    checkpoint

7.  [ ] Successfully ran forward pass in Transformers that gives
    identical output to original checkpoint

8.  [ ] Finished model tests in Transformers

9.  [ ] Successfully added Tokenizer in Transformers
Backward pass information: 
Retrieved documents:


===== Document 0 =====
Saving all activations from the for

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Based on the retrieved documents, it appears that the backward pass is generally slower than the forward  
pass in transformer model training. This is because the backward pass requires computing the gradients of the loss 
function with respect to the model's parameters, which can be computationally expensive.                           
                                                                                                                   
However, there are some techniques that can be used to speed up the backward pass, such as:                        
                                                                                                                   
*   Reversible transformer layers, which can be used to obtain the intermediate results of each layer during the   
backward pass                                                                                                      
*   Computing the feedforward operations by chunks, rather than on the whole batch                                 
*   Using activation checkpointing, which can help to reduce the memory overhead of the backward pass              
                                                                                                                   
<code>                                                                                                             
# The forward pass is generally faster than the backward pass in transformer model training                        
final_answer("forward pass")                                                                                       
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # The forward pass is generally faster than the backward pass in transformer model training                      
  final_answer("forward pass")                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: forward pass

[Step 2: Duration 1.45 seconds| Input tokens: 5,690 | Output tokens: 299]


Final answer:
forward pass


## Practical Applications of Agentic RAG

Agentic RAG systems can be applied to various use cases:

1. **Technical Documentation Assistance**: Help users navigate complex technical documentation
2. **Research Paper Analysis**: Extract and synthesize information from scientific papers
3. **Legal Document Review**: Find relevant precedents and clauses in legal documents
4. **Customer Support**: Answer questions based on product documentation and knowledge bases
5. **Educational Tutoring**: Provide explanations based on textbooks and learning materials

## Conclusion

Agentic RAG represents a significant advancement over traditional RAG pipelines. By combining the reasoning capabilities of LLM agents with the factual grounding of retrieval systems, we can build more powerful, flexible, and accurate information systems.

The approach we've demonstrated:
- Overcomes the limitations of single-step retrieval
- Enables more natural interactions with knowledge bases
- Provides a framework for continuous improvement through self-critique and query refinement

As you build your own Agentic RAG systems, consider experimenting with different retrieval methods, agent architectures, and knowledge sources to find the optimal configuration for your specific use case.

In [8]:
# To use Hugging Face's Inference API, you'll need to set up your API token.
# Go to https://huggingface.co/settings/tokens and create a new token.
# In Colab, click on the "🔑" icon in the left sidebar,
# add a new secret with the name `HF_TOKEN` and paste your token there.
# Then, uncomment and run the following line:
# from google.colab import userdata
# import os
# os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [9]:
from google.colab import userdata
import os
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [ ]:
# Load environment variables (including HF_TOKEN)
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

# Load the Hugging Face documentation dataset
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# Filter to include only Transformers documentation
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

# Convert dataset entries to Document objects with metadata
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

# Split documents into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Characters per chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],  # Priority order for splitting
)
docs_processed = text_splitter.split_documents(source_docs)

print(f"Knowledge base prepared with {len(docs_processed)} document chunks")
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        # Initialize the retriever with our processed documents
        self.retriever = BM25Retriever.from_documents(
            docs, k=10  # Return top 10 most relevant documents
        )

    def forward(self, query: str) -> str:
        """Execute the retrieval based on the provided query."""
        assert isinstance(query, str), "Your search query must be a string"

        # Retrieve relevant documents
        docs = self.retriever.invoke(query)

        # Format the retrieved documents for readability
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# Initialize our retriever tool with the processed documents
retriever_tool = RetrieverTool(docs_processed)
from smolagents import InferenceClientModel, CodeAgent

# Initialize the agent with our retriever tool
agent = CodeAgent(
    tools=[retriever_tool],
    model=InferenceClientModel(model_id="meta-llama/Llama-3.2-3B-Instruct"),
    # List of tools available to the agent
    #model=InferenceClientModel(),  # Default model "Qwen/Qwen2.5-Coder-32B-Instruct"
    max_steps=4,  # Limit the number of reasoning steps
    verbosity_level=2,  # Show detailed agent reasoning
)

# To use a specific model, you can specify it like this:

# Ask a question that requires retrieving information
question = "For a transformers model training, which is slower, the forward or the backward pass?"

# Run the agent to get an answer
agent_output = agent.run(question)

# Display the final answer
print("\nFinal answer:")
print(agent_output)

In [ ]:
pip install "smolagents[transformers]" torch accelerate bitsandbytes

In [ ]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from smolagents import Tool, CodeAgent
from smolagents import TransformersModel # <-- 1. استيراد TransformersModel

# تحميل مجموعة بيانات Hugging Face documentation
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# تصفية لتضمين وثائق Transformers فقط
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

# تحويل إدخالات مجموعة البيانات إلى كائنات Document مع بيانات وصفية
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

# تقسيم المستندات إلى أجزاء أصغر لاسترجاع أفضل
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # عدد الأحرف لكل جزء
    chunk_overlap=50,  # التداخل بين الأجزاء للحفاظ على السياق
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],  # ترتيب أولوية التقسيم
)
docs_processed = text_splitter.split_documents(source_docs)

print(f"تم تجهيز قاعدة المعرفة بـ {len(docs_processed)} جزء من المستندات")

class RetrieverTool(Tool):
    name = "retriever"
    description = "يستخدم البحث الدلالي لاسترداد أجزاء من وثائق transformers التي قد تكون الأكثر صلة للإجابة على استفسارك."
    inputs = {
        "query": {
            "type": "string",
            "description": "الاستعلام الذي سيتم تنفيذه. يجب أن يكون قريبًا دلاليًا من المستندات المستهدفة. استخدم صيغة التأكيد بدلاً من السؤال.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        # تهيئة المسترجع بمستنداتنا المعالجة
        self.retriever = BM25Retriever.from_documents(
            docs, k=10  # إرجاع أفضل 10 مستندات ذات صلة
        )

    def forward(self, query: str) -> str:
        """تنفيذ الاسترداد بناءً على الاستعلام المقدم."""
        assert isinstance(query, str), "يجب أن يكون استعلام البحث الخاص بك سلسلة نصية"

        # استرداد المستندات ذات الصلة
        docs = self.retriever.invoke(query)

        # تنسيق المستندات المستردة لسهولة القراءة
        return "\nالمستندات المستردة:\n" + "".join(
            [
                f"\n\n===== المستند {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# تهيئة أداة المسترجع الخاصة بنا بالمستندات المعالجة
retriever_tool = RetrieverTool(docs_processed)

# <-- 2. استبدال InferenceClientModel بـ TransformersModel
# ملاحظة: سيتم تنزيل النموذج وتخزينه مؤقتًا في المرة الأولى التي يتم فيها تشغيل الكود.
# يمكنك استبدال "meta-llama/Meta-Llama-3-8B-Instruct" بأي نموذج آخر يدعم توليد الكود من Hugging Face Hub.
local_model = TransformersModel(model_id="meta-llama/Llama-3.2-3B-Instruct")

# تهيئة العميل بأداة المسترجع الخاصة بنا
agent = CodeAgent(
    tools=[retriever_tool],
    model=local_model,  # <-- 3. استخدام النموذج المحلي
    max_steps=4,  # تحديد عدد خطوات التفكير
    verbosity_level=2,  # إظهار تفاصيل تفكير العميل
)

# طرح سؤال يتطلب استرداد معلومات
question = "For a transformers model training, which is slower, the forward or the backward pass?"

# تشغيل العميل للحصول على إجابة
agent_output = agent.run(question)

# عرض الإجابة النهائية
print("\nالإجابة النهائية:")
print(agent_output)


تم تجهيز قاعدة المعرفة بـ 14695 جزء من المستندات


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from smolagents import Tool, CodeAgent, TransformersModel

# 1. Load the Hugging Face documentation dataset
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# 2. Filter to include only Transformers documentation
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

# 3. Convert dataset entries to Document objects with metadata
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

# 4. Split documents into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Characters per chunk
    chunk_overlap=50,      # Overlap between chunks to maintain context
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],  # Priority order for splitting
)
docs_processed = text_splitter.split_documents(source_docs)

print(f"Knowledge base prepared with {len(docs_processed)} document chunks")

# 5. Define a tool for the agent to use for retrieving information
class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        # Initialize the retriever with our processed documents
        self.retriever = BM25Retriever.from_documents(
            docs, k=10  # Return top 10 most relevant documents
        )

    def forward(self, query: str) -> str:
        """Execute the retrieval based on the provided query."""
        assert isinstance(query, str), "Your search query must be a string"

        # Retrieve relevant documents
        docs = self.retriever.invoke(query)

        # Format the retrieved documents for readability
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# 6. Initialize the retriever tool with the processed documents
retriever_tool = RetrieverTool(docs_processed)

# 7. Initialize the local model using TransformersModel
# The model_kwargs dictionary is used to pass additional arguments directly
# to the .from_pretrained() method of the transformers library.
# `device_map="auto"` automatically distributes the model across available devices (GPU/CPU).
local_model = TransformersModel(
    model_id="meta-llama/Llama-3.2-3B-Instruct",
    device_map="auto" # Pass device_map directly
)

# 8. Initialize the agent with our retriever tool and the local model
agent = CodeAgent(
    tools=[retriever_tool],     # List of tools available to the agent
    model=local_model,          # Use the locally loaded model
    max_steps=4,                # Limit the number of reasoning steps
    verbosity_level=2,          # Show detailed agent reasoning
)

# 9. Ask a question that requires retrieving information
question = "For a transformers model training, which is slower, the forward or the backward pass?"

# 10. Run the agent to get an answer
agent_output = agent.run(question)

# 11. Display the final answer
print("\nFinal answer:")
print(agent_output)

### without api

In [2]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from smolagents import Tool, CodeAgent, TransformersModel

# 1. Load the Hugging Face documentation dataset
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# 2. Filter to include only Transformers documentation
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

# 3. Convert dataset entries to Document objects with metadata
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

# 4. Split documents into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Characters per chunk
    chunk_overlap=50,      # Overlap between chunks to maintain context
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],  # Priority order for splitting
)
docs_processed = text_splitter.split_documents(source_docs)

print(f"Knowledge base prepared with {len(docs_processed)} document chunks")

# 5. Define a tool for the agent to use for retrieving information
class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        # Initialize the retriever with our processed documents
        self.retriever = BM25Retriever.from_documents(
            docs, k=10  # Return top 10 most relevant documents
        )

    def forward(self, query: str) -> str:
        """Execute the retrieval based on the provided query."""
        assert isinstance(query, str), "Your search query must be a string"

        # Retrieve relevant documents
        docs = self.retriever.invoke(query)

        # Format the retrieved documents for readability
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# 6. Initialize the retriever tool with the processed documents
retriever_tool = RetrieverTool(docs_processed)

# 7. Initialize the local model using TransformersModel
# The model_kwargs dictionary is used to pass additional arguments directly
# to the .from_pretrained() method of the transformers library.
# `device_map="auto"` automatically distributes the model across available devices (GPU/CPU).
local_model = TransformersModel(
    model_id="meta-llama/Llama-3.2-3B-Instruct",
    device_map="auto" # Pass device_map directly
)

# 8. Initialize the agent with our retriever tool and the local model
agent = CodeAgent(
    tools=[retriever_tool],     # List of tools available to the agent
    model=local_model,          # Use the locally loaded model
    max_steps=4,                # Limit the number of reasoning steps
    verbosity_level=2,          # Show detailed agent reasoning
)

# 9. Ask a question that requires retrieving information
question = "For a transformers model training, which is slower, the forward or the backward pass?"

# 10. Run the agent to get an answer
agent_output = agent.run(question)

# 11. Display the final answer
print("\nFinal answer:")
print(agent_output)

Knowledge base prepared with 14695 document chunks


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Define variables                                                                                               
  forward_time = 0                                                                                                 
  backward_time = 0                                                                                                
                                                                                                                   
  # Assume a complex computation for the forward pass                                                              
  # This could involve multiple layers, attention mechanisms, and other computationally expensive operations       
  forward_time = 10  # seconds                                                                                     
                                                                                                                   
  # Assume a complex computation for the backward pass                                                             
  # This could involve multiple layers, attention mechanisms, and other computationally expensive operations       
  backward_time = 20  # seconds                                                                                    
                                                                                                                   
  # Print the results                                                                                              
  print("Forward pass time:", forward_time)                                                                        
  print("Backward pass time:", backward_time)                                                                      
                                                                                                                   
  # Compare the times                                                                                              
  if forward_time < backward_time:                                                                                 
      final_answer("The forward pass is slower.")                                                                  
  elif forward_time > backward_time:                                                                               
      final_answer("The backward pass is slower.")                                                                 
  else:                                                                                                            
      final_answer("Both passes are equally slow.")                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Forward pass time: 10
Backward pass time: 20

Final answer: The forward pass is slower.

[Step 1: Duration 4339.98 seconds| Input tokens: 2,001 | Output tokens: 290]


Final answer:
The forward pass is slower.
